    # Project 1 Minimizing Tracking Error

# We construct our own index by selecting 30 stocks with top market cap in technology sector
# The reasons for choosing the technology stock include:
# 1. High growing opportunity
# 2. High market capitalization
# 3. Actively traded
# 4. Great investments
# We simply use the equally-weighted approach to get the index price level. The time of the data starts from the 2000-12-31 to 2017-12-31
# The training data starts from 2000-12-31 to 2008-12-31 while the testing data starts from 2009-1-1 to 2017-12-31.
# We devide our dataset based on roughly two economy cycles

In [ ]:
# The code in this part is to get ready for the dataset
import pandas_datareader.data as web
import numpy as np
import pandas as pd  
import scipy.io as spio
#pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override() 
import datetime 
import matplotlib.pyplot as plt
import warnings
from scipy import stats
import itertools
warnings.filterwarnings("ignore")

def getDataBatch(tickers, startdate, enddate):
  def getData(ticker):
    return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
  datas = map(getData, tickers)
  return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))
get historical stock price data
Flag_downloadData = False
# define the time period 
start_dt = datetime.datetime(2000, 12, 31)
end_dt = datetime.datetime(2017, 12, 31)

tickers = ['AAPL', 'ADBE', 'AMAT', 'ANSS', 'ASML', 'CERN', 'CHKP', 'CSCO', 'CTSH', 'CTXS', 'FFIV', 'FISV', 'INTU', 'JKHY', 'LRCX', 'MCHP', 'MRVL', 'MSFT', 'MXIM', 'NOK', 'NTAP', 'NVDA', 'ORCL', 'QCOM', 'SNPS', 'SYMC', 'TSM', 'TTWO', 'VRSN', 'XLNX']
# Marketcap-weighted
# wts_Index = pd.Series([0.293082, 0.266411, 0.066398, 0.064882, 0.061179, 0.039640, 0.037962, 0.021618, 0.017907, 0.012936, 0.010914, 0.010704, 0.010243, 0.007324, 0.007010, 0.006689, 0.006155, 0.006048, 0.005852, 0.005632, 0.004735, 0.004723, 0.004713, 0.004401, 0.004303, 0.004135, 0.004012, 0.003501, 0.003468, 0.003424])
# Equally-weighted
wts_Index = pd.Series([1/30] * 30)

if Flag_downloadData:
    
    stock_data = getDataBatch(tickers, start_dt, end_dt)
    # Isolate the `Adj Close` values and transform the DataFrame
    daily_close_px = stock_data.reset_index().pivot(index='Date', columns='Ticker', values='Adj Close')
    # Marketcap-weighted
    #index_close_px = pd.DataFrame(np.dot(daily_close_px.iloc[:,0:30], [0.293082, 0.266411, 0.066398, 0.064882, 0.061179, 0.039640, 0.037962, 0.021618, 0.017907, 0.012936, 0.010914, 0.010704, 0.010243, 0.007324, 0.007010, 0.006689, 0.006155, 0.006048, 0.005852, 0.005632, 0.004735, 0.004723, 0.004713, 0.004401, 0.004303, 0.004135, 0.004012, 0.003501, 0.003468, 0.003424]), index = daily_close_px.index.values)
    # Equally-weighted
    index_close_px = pd.DataFrame(np.dot(daily_close_px.iloc[:,0:30], [1/30] * 30), index = daily_close_px.index.values)
    index_close_px.columns = ['Index']
    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('Price.xlsx', engine='xlsxwriter')
    daily_close_px.to_excel(writer, sheet_name='PriceStock',startrow=0, startcol=0, header=True, index=True)
    index_close_px.to_excel(writer, sheet_name='PriceIndex',startrow=0, startcol=0, header=True, index=True)
else:
    daily_close_px = pd.read_excel('Price.xlsx', sheet_name='PriceStock',
                    header=0, index_col = 0)
    index_close_px = pd.read_excel('Price.xlsx', sheet_name='PriceIndex',
                    header=0, index_col = 0)

plt.plot(index_close_px)
plt.ylabel('Index Level')
plt.xlabel('Years')
plt.title('Steve Tech 30 Index')
plt.savefig('Steve Tech 30 Index')
# calculate the historical daily returns of each stocks and the index
# daily_pct_change = daily_close_px.pct_change().dropna()

# Next we want to construct a ETF fund by using the top 10 to 20 stock to track the index and minimize the tracking error.
# We firstly use the training data to get the optimized weights and then use the optimized weights to claculate the tracking error of the testing data

In [ ]:
# The code in this part shows the step of calculating the covariance matrix of the training data
# Choose top 20 stocks to build the portfolio and track the index
# Returns
ret_AllStock = daily_close_px.pct_change().dropna()
ret_Idx = index_close_px.pct_change().dropna()
ret_Idx.columns = ['Return']

# Scale return data by a factor. It seems that the optimizer fails when the values are too close to 0
scale = 1
ret_AllStock = ret_AllStock*scale
ret_Idx = ret_Idx*scale

num_periods, num_stock = ret_AllStock.shape

# Calulate Covariance Matrix
#
# load module with utility functions, including optimization 
import risk_opt_2Student as riskopt 
def tracking_error(wts_active,cov):
    TE = np.sqrt(np.transpose(wts_active)@cov@wts_active)
    return TE
lamda = 0.94

# Train the model (Training Set: 2000-12-31, 2008-12-31)
ret_AllStock = ret_AllStock.iloc[0:2010,:]
ret_Idx = ret_Idx.iloc[0:2010,:]

#ret_AllStock = ret_AllStock.iloc[0:1935,:]
#ret_Idx = ret_Idx.iloc[0:1935,:]

num_periods, num_stock = ret_AllStock.shape
# vol of the assets 
vols = ret_AllStock.std()
rets_mean = ret_AllStock.mean()
# demean the returns
ret_AllStock = ret_AllStock - rets_mean

# var_ewma calculation of the covraiance using the function from module risk_opt.py
var_ewma = riskopt.ewma_cov(ret_AllStock, lamda)
var_ewma_annual = var_ewma*252 #Annualize
# take only the covariance matrix for the last date, which is the forecast for next time period
cov_end = var_ewma[-1,:]
#
cov_end_annual = cov_end*252 #Annualize
std_end_annual = np.sqrt(np.diag(cov_end))*np.sqrt(252)
# calculate the correlation matrix
corr = ret_AllStock.corr()

In [ ]:
# The code in this part shows the optimized weights and the minimized tracking error of the traing data.
# We use the top 10 to 20 weighted stock to get the results
# tracking error optimization
#
wts_active = np.zeros([num_stock,1])
wts_active[0] = 0.05
wts_active[-1] = -0.05
#np.transpose(wts_active)@cov_end@wts_active
TE1 = tracking_error(wts_active,cov_end)
# The code above is only an example of calculating TE
#
# Test case - minize TE to zero should produce a fund with wts like those of the index

num_topwtstock_2include = 10 #bad with 13, 16, 17, 18, 19, 20, 21,
# only the top weight stocks + no shorting 
b1a_ = [(0.0,1.0) for i in range(num_topwtstock_2include)] 
# exclude bottom weighted stocks
b1b_ = [(0.0,0.0) for i in range(num_topwtstock_2include,num_stock)] 
b1_ = b1a_ + b1b_ # combining the constraints
# b1_[num_topwtstock_2include:-1] = (0.0,0.0)
c1_ = ({'type':'eq', 'fun': lambda W: sum(W)-1. })  # Sum of active weights = 100%
# Calling the optimizer
wts_min_trackingerror1 = riskopt.opt_min_te(wts_Index, cov_end_annual, b1_, c1_)
# calc TE achieved
wts_active1 = wts_min_trackingerror1 - wts_Index
TE_optimized1 = tracking_error(wts_active1,cov_end)
print('{0} top weighted stock replication TE = {1:5.2f} bps'.format(num_topwtstock_2include, TE_optimized1*10000))

# looping through number of stocks and save the history of TEs
num_stock_b = 10
num_stock_e = 21
numstock_2use = range(num_stock_b,num_stock_e)
wts_active_hist = np.zeros([len(numstock_2use), num_stock])
TE_hist = np.zeros([len(numstock_2use), 1])
count = 0
# TE_hist is the minimized tracking error choosing different number of stocks

for i in numstock_2use:
    # only the top weight stocks + no shorting 
    b1_c_a_ = [(0.0,1.0) for j in range(i)] 
    # exclude bottom weighted stocks
    b1_c_b_ = [(0.0,0.0) for j in range(i,num_stock)] 
    b1_curr_ = b1_c_a_ + b1_c_b_
    wts_min_curr = riskopt.opt_min_te(wts_Index, cov_end_annual, b1_curr_, c1_)
    wts_active_hist[count,:] = wts_min_curr.transpose()
    TE_optimized_c = tracking_error(wts_min_curr-wts_Index,cov_end)
    TE_hist[count,:] = TE_optimized_c*10000# in bps
    count = count+1
    
    del b1_curr_, b1_c_a_, b1_c_b_,TE_optimized_c,wts_min_curr

# Here comes to the interesting part of the project.
# We do not think it is reasonable to choose the top weighted stock ETF to track the equally-weighted index.
# We guess that it might be much more efficient to randomly choose the stock but with fewer number of stocks
# So we use the itertools.combinations to construct randomly chosen portfolio. # For example, if we randomly choose 2 stocks from 30 using the training data, there will be 435 scenario of combinations. 
# We calculate the optimized weight for each scenario and find the optimized weight matrix of the scenario which has smallest tracking error. 
# The optimized weight matrix we find above will be used directly to calculate the testing data tracking error.
# The code in the following cell illustrate an example of 5 randomly chosen stock from which we get the optimized weight matrix and the tracking error

In [ ]:
Not using the TOP stocks
#Try picking the stocks randomly
    
# which stocks should use?
Flag_downloadData2 = False
if Flag_downloadData2:
    #TE_hist_own = np.zeros([435, 1])
    #wts_active_hist_own = np.zeros([435, 30])
    TE_hist_own = np.zeros([142506, 1])
    wts_active_hist_own = np.zeros([142506, 30])
    count = 0
    
    # Iterate each combination
    for i in itertools.combinations(tickers, 5):
    
        # generate the constraints
        b1_own = [(0.0,0.0) for n in range(30)]
        for j in i:
            colIndex = ret_AllStock.columns.get_loc(j)
            b1_own[colIndex] = (0.0,1.0)
        
        wts_min_curr = riskopt.opt_min_te(wts_Index, cov_end_annual, b1_own, c1_)
        wts_active_hist_own[count,:] = wts_min_curr.transpose()
        TE_optimized_c = tracking_error(wts_min_curr-wts_Index,cov_end)
        TE_hist_own[count,:] = TE_optimized_c*10000# in bps
        count = count+1
        
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('5Stocks.xlsx', engine='xlsxwriter')
    pd.DataFrame(wts_active_hist_own).to_excel(writer, sheet_name='wts',startrow=0, startcol=0, header=True, index=True)
    pd.DataFrame(TE_hist_own).to_excel(writer, sheet_name='TE',startrow=0, startcol=0, header=True, index=True)

else:
    wts_active_hist_own = pd.read_excel('5Stocks.xlsx', sheet_name='wts',
                    header=0, index_col = 0)
    TE_hist_own = pd.read_excel('5Stocks.xlsx', sheet_name='TE',
                    header=0, index_col = 0)

min_TE = TE_hist_own.min()
min_Loc = np.where(TE_hist_own == min_TE)[0]
optimized_comb = np.array(wts_active_hist_own)[min_Loc]
print(min_TE)

# We use the training dataset to get our own ETF fund of 5 randomly chosen stock and compare the minimized TE to that of top weighted stocks. 
# We find the minimized TE of the randomly chosen ETF is still smaller than the top 14 weighted ETF
# The result interprets the investment efficiency improvement by only holding a ETF of 5 stocks which has a better TE than the ETF of 14 stocks.
# However, the time efficiency of our algorithms is quite low. To illustrate, if we want to randomly choose 10 stocks to reach a much smaller TE, we will spend more than a week to get the result from our algorithms.

# In the following parts, we start to calculate the TE for the testing data of the top weighted stocks and randomly chosen stocks respectively

In [ ]:
# Test the top weighted ETF model using test set  (Test Set: 2008-12-31, 2017-12-31)
# Firstly is to calculate the covariance matrix for the test dataset
# generate the return for test dataset
ret_AllStock = daily_close_px.pct_change().dropna()
ret_Idx = index_close_px.pct_change().dropna()
ret_Idx.columns = ['Return']
num_periods, num_stock = ret_AllStock.shape
test_ret_AllStock = ret_AllStock.iloc[2010:,:]
test_ret_Idx = ret_Idx.iloc[2010:,:]
num_periods, num_stock = test_ret_AllStock.shape
# calculate the vol and mean 
vols = test_ret_AllStock.std()
rets_mean = test_ret_AllStock.mean()
# demean the returns
test_ret_AllStock = test_ret_AllStock - rets_mean
# var_ewma calculation of the covraiance using the function from module risk_opt.py
var_ewma = riskopt.ewma_cov(test_ret_AllStock, lamda)
var_ewma_annual = var_ewma*252 #Annualize
# take only the covariance matrix for the last date, which is the forecast for next time period
test_cov_end = var_ewma[-1,:]
test_cov_end_annual = test_cov_end*252 #Annualize
test_std_end_annual = np.sqrt(np.diag(test_cov_end))*np.sqrt(252)
# calculate the correlation matrix
test_corr = test_ret_AllStock.corr()

In [ ]:
# Calculate the TE for test set (Top 10 stocks - Top 20 stocks)
# use the training weight to calculate the new TE
test_TE_hist = np.zeros([len(numstock_2use), 1])
count = 0
for i in numstock_2use:
    # only the top weight stocks + no shorting 
    b1_c_a_ = [(0.0,1.0) for j in range(i)] 
    # exclude bottom weighted stocks
    b1_c_b_ = [(0.0,0.0) for j in range(i,num_stock)] 
    b1_curr_ = b1_c_a_ + b1_c_b_
    # using the weights from training covariance matrix
    wts_min_curr = riskopt.opt_min_te(wts_Index, cov_end_annual, b1_curr_, c1_)
    test_TE_optimized_c = tracking_error(wts_min_curr-wts_Index,test_cov_end)
    test_TE_hist[count,:] = test_TE_optimized_c*10000# in bps
    count = count+1

    del b1_curr_, b1_c_a_, b1_c_b_,test_TE_optimized_c,wts_min_curr

In [ ]:
# Calculate the TE for random five model
# testing our own model using the latter part of our dataset
own_test_TE = tracking_error(pd.Series(optimized_comb.transpose()[:,0])-wts_Index,test_cov_end)*10000
print(own_test_TE)

# In the following part, we will make some plots to better illustrate our results

In [ ]:
#  Plot bars of weights
#
figure_count = 1
# ---  create plot of weights fund vs benchmark(top weighted model)---
plt.figure(figure_count)
figure_count = figure_count+1
fig, ax = plt.subplots(figsize=(18,10))
index = np.arange(len(wts_Index))
bar_width = 0.35
opacity = 0.8
 
rects1 = plt.bar(index, wts_Index, bar_width,
                 alpha=opacity,
                 color='b',
                 label='Index Weight')
 
rects2 = plt.bar(index + bar_width, wts_min_trackingerror1, bar_width,
                 alpha=opacity,
                 color='g',
                 label='ETF fund Weight')
 
plt.xlabel('Ticker', fontsize=18)
plt.ylabel('Weights', fontsize=18)
plt.xticks(index + bar_width, (tickers), fontsize=12)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=18)
plt.legend(fontsize=20)
plt.title('Steve Tech ETF 10 vs.Benchmark',fontsize=18) 
plt.tight_layout()
plt.show()
plt.savefig('Steve Tech ETF 10 vs Benchmark.jpeg')
#this bar chart only show the optimized wieght of ETF of 10 stock compared to the index

#------plot TE as a function of number of stocks(top weighted model) -------------
plt.figure(figure_count)
figure_count = figure_count+1
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(range(num_stock_b,num_stock_e), TE_hist, 'b')
plt.xlabel('Number of stocks in ETF', fontsize=18)
plt.ylabel('Optimized Tracking Error (bps)', fontsize=18)
plt.title('Steve Tech 30 ETF', fontsize=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
plt.title("Training Data TE", fontsize=18)

In [ ]:
# combine the TE plot of traing dataset with the training data set(top-weighted model)
plt.figure(figure_count)
figure_count = figure_count+1
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(range(num_stock_b,num_stock_e), TE_hist, color='black',label="Training Data")
plt.plot(range(num_stock_b,num_stock_e),test_TE_hist, color='red',label="Testing Data")
plt.xlabel('Number of stocks in ETF', fontsize=18)
plt.ylabel('Optimized Tracking Error (bps)', fontsize=18)
plt.title('Steve Tech 30 ETF', fontsize=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
plt.title("Training Data TE vs Testing Data TE", fontsize=18)
plt.legend(fontsize=18)
plt.savefig("Training Data TE vs Testing Data TE")

In [ ]:
#do the bar chart weight using our own random chosen 5 stock
plt.figure(figure_count)
figure_count = figure_count+1
fig, ax = plt.subplots(figsize=(18,10))
index = np.arange(len(wts_Index))
bar_width = 0.35
opacity = 0.8
 
rects1 = plt.bar(index, wts_Index, bar_width,
                 alpha=opacity,
                 color='b',
                 label='Index Weight')
 
rects2 = plt.bar(index + bar_width, pd.Series(optimized_comb.transpose()[:,0]), bar_width,
                 alpha=opacity,
                 color='g',
                 label='Steve Tech 5 ETF Weight')
 
plt.xlabel('Ticker', fontsize=18)
plt.ylabel('Weights', fontsize=18)
plt.xticks(index + bar_width, (tickers), fontsize=12)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=18)
plt.legend(fontsize=20)
plt.title('Steve Tech ETF 5 vs Benchmark jpeg',fontsize=18) 
plt.tight_layout()
plt.show()
plt.savefig('Steve Tech ETF 5 vs Benchmark.jpeg')

In [ ]:
#compare the TE traing data of top weighted vs randomly chosen
min_TE_list=[52.252179]*11
plt.figure(figure_count)
figure_count = figure_count+1
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(range(num_stock_b,num_stock_e), TE_hist, color='black',label="Top Weighted Stock")
plt.plot(range(num_stock_b,num_stock_e),min_TE_list, color='red',label="Steve Tech 5")
plt.xlabel('Number of stocks in ETF', fontsize=18)
plt.ylabel('Optimized Tracking Error (bps)', fontsize=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
plt.title("Training Data", fontsize=18)
plt.legend(fontsize=18)
plt.savefig("Training Data_Steve Tech 5 vs Top weighted")

In [ ]:
#compare the TE testing data of top weighted vs randomly chosen
own_test_TE_list=[35.45713253022448]*11
plt.figure(figure_count)
figure_count = figure_count+1
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(range(num_stock_b,num_stock_e), test_TE_hist, color='black',label="Top Weighted Stock")
plt.plot(range(num_stock_b,num_stock_e),own_test_TE_list, color='red',label="Steve Tech 5")
plt.xlabel('Number of stocks in ETF', fontsize=18)
plt.ylabel('Optimized Tracking Error (bps)', fontsize=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
plt.title("Testing Data", fontsize=18)
plt.legend(fontsize=18)
plt.savefig("Testing Data_Steve Tech 5 vs Top weighted")

# More to think about:
# Is it a more efficient algorithsm to randomly construct the ETF?
# What is the reason for a better result from testing data?
# Is it more optimal to do the dynamic tracking error?